In [112]:
using NetPricing, JuMP, Gurobi

# Import a problem from a file
file = "../tmp/000000-000000-g40-05-P.json"
#file = "../data/from_github/problems/paper/g40-05.json"
prob = read_problem(file)

# Preprocess the problem for each commodity
pprobs = preprocess(prob, maxpaths = 1000)

# Create a model
model, forms = std_model(pprobs)
# model, forms = pastd_model(pprobs)
# model, forms = vf_model(pprobs)
# model, forms = pvf_model(pprobs)

# Add strong bilevel feasibility (optional, only available for pastd and pvf models)
#add_strong_bf_cuts(model, forms, maxpairs=10000, commpairs=100)

# Solve the model
optimize!(model)

# Extract the result
tvals = value.(model[:t]);                   # The prices t
# Prend quelques seconde pour atteindre l'optimalite
# total profit : 85073
#

MINH-MODEL
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 24.04 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2181 rows, 1697 columns and 6442 nonzeros
Model fingerprint: 0x752a9b10
Variable types: 1450 continuous, 247 integer (247 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 3e+03]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e-10, 1e+02]
Presolve removed 756 rows and 719 columns
Presolve time: 0.02s
Presolved: 1425 rows, 978 columns, 4587 nonzeros
Variable types: 706 continuous, 272 integer (272 binary)

Root relaxation: objective 9.206241e+04, 781 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 92062.4052   

In [113]:
k = 6
primal_repr = primal(forms[k])              # Primal representation
dual_repr = NetPricing.dual(forms[k])       # Dual representation
prob_k = problem(primal_repr)               # Preprocessed problem of forms[k]

primal_obj = value(primal_repr.primalobj)   # Primal objective: c' x[k]
dual_obj = value(dual_repr.dualobj)         # Dual objective: b' λ[k]

xvals = value.(primal_repr.x)               # Arc selections x[k]
#zvals = value.(primal_repr.z)               # Path selections z[k] (only for primal-path)
λvals = value.(dual_repr.λ)                 # Dual prices λ[k] (only for dual-arc)
b = NetPricing.sourcesink_vector(prob_k)    # vector b source sink

# Maps from indices of the preprocessed problem to those of the base problem
Amap = used_arcs(prob_k)
Vmap = used_nodes(prob_k)
println()
# Vrevmap = NetPricing.revmap(Vmap, length(nodes(prob_k)))
# Arevmap = NetPricing.revmap(Amap, length(arcs(prob_k)))

# Primal and dual objectives must satisfy: primal_obj + t' x[k] == dual_obj
a1_k = tolled_arcs(prob_k)                  # List of tolled arcs of prob_k
@assert(dual_obj - primal_obj ≈ sum(tvals[Amap[a]] .* xvals[a] for a in a1_k))

In [114]:
println("Problem info")
println(prob)

println()
println("USER k=",k)
println(prob_k)
println("origin\t\t\t\t\t|\t",NetPricing.orig(prob_k))
println("destination\t\t\t\t|\t",NetPricing.dest(prob_k))
println()
println("Dual objective b' λ[k]\t\t\t|\t", dual_obj)
println("Primal objective c' x[k]\t\t|\t", primal_obj)
println("Profit from user b' λ[k] - c' x[k]\t|\t", dual_obj-primal_obj)
println("-----------------------------------------------------------\n")


println("λvals")
println("length\t\t\t\t\t|\t", length(λvals))
println("vector\t\t\t\t\t|\t", λvals)
println()
println("Vertices used index Vmap")
println("length\t\t\t\t\t|\t", length(Vmap))
println("vector\t\t\t\t\t|\t", Vmap)
println("-----------------------------------------------------------\n")

println("Arcs used xvals")
println("length\t\t\t\t\t|\t", length(xvals))
println("vector\t\t\t\t\t|\t", xvals)
println()
println("Arc index Amap")
println("length\t\t\t\t\t|\t", length(Amap))
println("vector\t\t\t\t\t|\t", Amap)
println()
println("Arcs used index")
println("Amap[xvals .== 1]\t\t\t|\t" , Amap[xvals .== 1.0])

println("-----------------------------------------------------------\n")


Problem info
Problem with {60 nodes, 206 arcs (42 tolled), 39 commodities}

USER k=6
Path-processed problem for k = 9 with {41 nodes, 66 arcs (17 tolled), 53 paths}
origin					|	3
destination				|	35

Dual objective b' λ[k]			|	253.0
Primal objective c' x[k]		|	134.0
Profit from user b' λ[k] - c' x[k]	|	119.0
-----------------------------------------------------------

λvals
length					|	41
vector					|	[182.0, 231.0, 253.0, 194.0, 177.0, 196.0, 231.0, 223.0, 176.0, 170.0, 183.0, 196.0, 204.0, 141.0, 162.0, 168.0, 168.0, 190.0, 94.0, 114.0, 133.0, 150.0, 169.64952292644648, 59.0, 81.0, 100.0, 127.0, 108.64952292644647, 45.0, 78.0, 90.0, 100.0, 95.64952292644647, 33.0, 0.0, 57.0, 79.0, 95.0, 18.0, 31.0, 44.0]

Vertices used index Vmap
length					|	41
vector					|	[17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59]
-----------------------------------------------------------

A

In [89]:
# Commentaire 
# les problemes forms[k] pour k=3 retourne celui pour k=5 (il y a suppression de problemes)


In [37]:
arcs(prob_k)

33-element Vector{ProblemArc}:
 Tolled arc 1 => 5 with cost 9.0
 Tolled arc 2 => 1 with cost 4.0
 Toll-free arc 3 => 2 with cost 12.0
 Tolled arc 4 => 8 with cost 5.0
 Tolled arc 5 => 4 with cost 13.0
 Toll-free arc 6 => 3 with cost 22.0
 Toll-free arc 6 => 11 with cost 8.0
 Toll-free arc 7 => 12 with cost 18.0
 Toll-free arc 8 => 7 with cost 10.0
 Tolled arc 8 => 13 with cost 7.0
 Toll-free arc 9 => 8 with cost 21.0
 Toll-free arc 9 => 14 with cost 13.0
 Toll-free arc 10 => 9 with cost 35.0
 ⋮
 Tolled arc 16 => 21 with cost 10.0
 Tolled arc 17 => 22 with cost 6.0
 Tolled arc 18 => 17 with cost 5.0
 Tolled arc 18 => 23 with cost 11.0
 Toll-free arc 19 => 18 with cost 6.0
 Toll-free arc 19 => 24 with cost 35.0
 Toll-free arc 20 => 19 with cost 13.0
 Toll-free arc 20 => 25 with cost 18.0
 Toll-free arc 21 => 20 with cost 35.0
 Toll-free arc 23 => 22 with cost 27.0
 Toll-free arc 24 => 23 with cost 19.0
 Toll-free arc 25 => 24 with cost 20.0

In [40]:
println([Amap[a] for a in a1_k])

[42, 44, 56, 58, 74, 85, 92, 103, 106, 108, 110]


In [44]:
# Vrevmap = NetPricing.revmap(Vmap, length(nodes(prob_k)))
# Arevmap = NetPricing.revmap(Amap, length(arcs(prob_k)));